# Notizbuch für den 'data' Endpunkt & Spezifikation von 'job=true'
Initial ist 'job=false' gesetzt, daher muss dieser Parameter bei der Abfrage eins zu großen Datensatzes angepasst werden.
Der User erhält über einen Input die Möglichkeit, den Job anzustoßen.
Dessen Status wird über den catalogue/jobs Endpunkt überprüft.
Final wird das gecachte resultfile ausgegeben.

In [1]:
import logging
from pygenesis.http_helper import get_data_from_endpoint

In [2]:
# set logging level to INFO to receive more detailed program response
logging.basicConfig(level=logging.INFO)

endpoint = 'data'
method = 'tablefile'
params = {'name':'42151-0001',
          'area':'all',
          'timeslices':20,
          'job':'false',
          'language':'de',}
response = get_data_from_endpoint(endpoint=endpoint, method=method, params=params)

ReadTimeout: HTTPSConnectionPool(host='www-genesis.destatis.de', port=443): Read timed out. (read timeout=15)

In [3]:
print(response)

<Response [42]>


In [2]:
endpoint = 'data'
method = 'tablefile'
params = {'name':'42151-0005',
          'area':'all',
          'timeslices':20,
          #'sortcriterion': 'Auftragstyp',
          'job':'false',
          'language':'de',}
response = load_data(endpoint=endpoint, method=method, params=params)
#initial_response = response.json()

Die Daten sind zu groß um direkt abgerufen zu werden.Es muss ein Job gestartet werden, der einige Sekunden braucht,um die Daten bereitzustellen und abzurufen.
 Sollen wir einen Job starten?
 Ja/Nein:


98
inner_loop
99
{'Ident': {'Service': 'data', 'Method': 'tablefile'}, 'Status': {'Code': 99, 'Content': 'Der Bearbeitungsauftrag wurde erstellt. Die Tabelle kann in Kürze als Ergebnis mit folgendem Namen abgerufen werden: 42151-0005_618384492', 'Type': 'Information'}, 'Parameter': {'username': '********************', 'password': '********************', 'name': '42151-0005', 'area': 'Alle', 'compress': 'false', 'transpose': 'false', 'startyear': '', 'endyear': '', 'timeslices': '20', 'regionalvariable': '', 'regionalkey': '', 'classifyingvariable1': '', 'classifyingkey1': '', 'classifyingvariable2': '', 'classifyingkey2': '', 'classifyingvariable3': '', 'classifyingkey3': '', 'format': 'csv', 'job': 'true', 'stand': '01.01.1970 01:00', 'language': 'de'}, 'Object': None, 'Copyright': '© Statistisches Bundesamt (Destatis), 2022'}
99 {'Ident': {'Service': 'data', 'Method': 'tablefile'}, 'Status': {'Code': 99, 'Content': 'Der Bearbeitungsauftrag wurde erstellt. Die Tabelle kann in Kürze al

erfolgreich (Mindestens ein Parameter enthält ungültige Werte. Er wurde angepasst, um den Service starten zu können.: sortcriterion)


22
{'Ident': {'Service': 'catalogue', 'Method': 'jobs'}, 'Status': {'Code': 22, 'Content': 'erfolgreich (Mindestens ein Parameter enthält ungültige Werte. Er wurde angepasst, um den Service starten zu können.: sortcriterion)', 'Type': 'Warnung'}, 'Parameter': {'username': '********************', 'password': '********************', 'selection': '*42151-0005_618384492*', 'searchcriterion': 'Code', 'sortcriterion': 'Auftragstyp', 'type': 'Alle', 'pagelength': '100', 'language': 'de', 'area': 'Benutzer'}, 'List': [{'Content': 'Werteabruf', 'Date': '19.09.2022', 'Time': '22:13:00', 'State': 'Wartet', 'Code': '42151-0005_618384492'}], 'Copyright': '© Statistisches Bundesamt (Destatis), 2022'}
{'Ident': {'Service': 'catalogue', 'Method': 'jobs'}, 'Status': {'Code': 22, 'Content': 'erfolgreich (Mindestens ein Parameter enthält ungültige Werte. Er wurde angepasst, um den Service starten zu können.: sortcriterion)', 'Type': 'Warnung'}, 'Parameter': {'username': '********************', 'password'

erfolgreich (Mindestens ein Parameter enthält ungültige Werte. Er wurde angepasst, um den Service starten zu können.: sortcriterion)


...

erfolgreich (Mindestens ein Parameter enthält ungültige Werte. Er wurde angepasst, um den Service starten zu können.: sortcriterion)


22
{'Ident': {'Service': 'catalogue', 'Method': 'jobs'}, 'Status': {'Code': 22, 'Content': 'erfolgreich (Mindestens ein Parameter enthält ungültige Werte. Er wurde angepasst, um den Service starten zu können.: sortcriterion)', 'Type': 'Warnung'}, 'Parameter': {'username': '********************', 'password': '********************', 'selection': '*42151-0005_618384492*', 'searchcriterion': 'Code', 'sortcriterion': 'Auftragstyp', 'type': 'Alle', 'pagelength': '100', 'language': 'de', 'area': 'Benutzer'}, 'List': [{'Content': 'Werteabruf', 'Date': '19.09.2022', 'Time': '22:13:00', 'State': 'Aktiv', 'Code': '42151-0005_618384492'}], 'Copyright': '© Statistisches Bundesamt (Destatis), 2022'}
{'Ident': {'Service': 'catalogue', 'Method': 'jobs'}, 'Status': {'Code': 22, 'Content': 'erfolgreich (Mindestens ein Parameter enthält ungültige Werte. Er wurde angepasst, um den Service starten zu können.: sortcriterion)', 'Type': 'Warnung'}, 'Parameter': {'username': '********************', 'password':

erfolgreich (Mindestens ein Parameter enthält ungültige Werte. Er wurde angepasst, um den Service starten zu können.: sortcriterion)


22
{'Ident': {'Service': 'catalogue', 'Method': 'jobs'}, 'Status': {'Code': 22, 'Content': 'erfolgreich (Mindestens ein Parameter enthält ungültige Werte. Er wurde angepasst, um den Service starten zu können.: sortcriterion)', 'Type': 'Warnung'}, 'Parameter': {'username': '********************', 'password': '********************', 'selection': '*42151-0005_618384492*', 'searchcriterion': 'Code', 'sortcriterion': 'Auftragstyp', 'type': 'Alle', 'pagelength': '100', 'language': 'de', 'area': 'Benutzer'}, 'List': [{'Content': 'Werteabruf', 'Date': '19.09.2022', 'Time': '22:13:00', 'State': 'Fertig', 'Code': '42151-0005_618384492'}], 'Copyright': '© Statistisches Bundesamt (Destatis), 2022'}
{'Ident': {'Service': 'catalogue', 'Method': 'jobs'}, 'Status': {'Code': 22, 'Content': 'erfolgreich (Mindestens ein Parameter enthält ungültige Werte. Er wurde angepasst, um den Service starten zu können.: sortcriterion)', 'Type': 'Warnung'}, 'Parameter': {'username': '********************', 'password'

DestatisStatusError: Die Tabelle ist zu gross, um im Dialog abgerufen zu werden. Bitte nutzen sie den Webservice mit dem Parameter auftrag=true um eine Verarbeitung im Hintergrund zu starten.

In [4]:
endpoint = 'data'
method = 'tablefile'
params = {'name':'42151-0005',
          'area':'all',
          'timeslices':20,
          'sortcriterion': 'Auftragstyp',
          'job':'false',
          'language':'de',}
response = get_data_from_endpoint(endpoint=endpoint, method=method, params=params)
initial_response = response.json()

Die Daten sind zu groß um direkt abgerufen zu werden.Es muss ein Job gestartet werden, der einige Sekunden braucht,um die Daten bereitzustellen und abzurufen.
 Sollen wir einen Job starten?
 Ja/Nein:


98
{'Ident': {'Service': 'data', 'Method': 'tablefile'}, 'Status': {'Code': 98, 'Content': 'Die Tabelle ist zu gross, um im Dialog abgerufen zu werden. Bitte nutzen sie den Webservice mit dem Parameter auftrag=true um eine Verarbeitung im Hintergrund zu starten.', 'Type': 'Fehler'}, 'Parameter': {'username': '********************', 'password': '********************', 'name': '42151-0005', 'area': 'Alle', 'compress': 'false', 'transpose': 'false', 'startyear': '', 'endyear': '', 'timeslices': '20', 'regionalvariable': '', 'regionalkey': '', 'classifyingvariable1': '', 'classifyingkey1': '', 'classifyingvariable2': '', 'classifyingkey2': '', 'classifyingvariable3': '', 'classifyingkey3': '', 'format': 'csv', 'job': 'false', 'stand': '01.01.1970 01:00', 'language': 'de'}, 'Object': None, 'Copyright': '© Statistisches Bundesamt (Destatis), 2022'}


DestatisStatusError: Die Tabelle ist zu gross, um im Dialog abgerufen zu werden. Bitte nutzen sie den Webservice mit dem Parameter auftrag=true um eine Verarbeitung im Hintergrund zu starten.

In [77]:
initial_response

{'Ident': {'Service': 'data', 'Method': 'tablefile'},
 'Status': {'Code': 98,
  'Content': 'Die Tabelle ist zu gross, um im Dialog abgerufen zu werden. Bitte nutzen sie den Webservice mit dem Parameter auftrag=true um eine Verarbeitung im Hintergrund zu starten.',
  'Type': 'Fehler'},
 'Parameter': {'username': '********************',
  'password': '********************',
  'name': '42153-0001',
  'area': 'Alle',
  'compress': 'false',
  'transpose': 'false',
  'startyear': '',
  'endyear': '',
  'timeslices': '20',
  'regionalvariable': '',
  'regionalkey': '',
  'classifyingvariable1': '',
  'classifyingkey1': '',
  'classifyingvariable2': '',
  'classifyingkey2': '',
  'classifyingvariable3': '',
  'classifyingkey3': '',
  'format': 'csv',
  'job': 'false',
  'stand': '01.01.1970 01:00',
  'language': 'de'},
 'Object': None,
 'Copyright': '© Statistisches Bundesamt (Destatis), 2022'}

In [78]:
initial_response.get('Status')

{'Code': 98,
 'Content': 'Die Tabelle ist zu gross, um im Dialog abgerufen zu werden. Bitte nutzen sie den Webservice mit dem Parameter auftrag=true um eine Verarbeitung im Hintergrund zu starten.',
 'Type': 'Fehler'}

Der Status ist vom Typ Fehler, da die Tabelle zu groß ist & mittels eines Jobs abgerufen werden muss. Wir stellen somit erneut den gleichen Funktionsaufruf, diese mal aber mit 'job=true'.

In [3]:
endpoint = 'data'
method = 'tablefile'
params = {'name':'42151-0005',
          'area':'all',
          'timeslices':20,
          #'sortcriterion': 'Auftragstyp',
          #'job':'false',
          'language':'de',}

from pygenesis.config import load_config

config = load_config()
url = f"{config['GENESIS API']['base_url']}{endpoint}/{method}"

#if initial_response.get('Status').get('Code') == 98:
params |= {'job':'true'}
response = requests.get(url, params=params, timeout=(5, 15))
print(response, response.text)
response.encoding = "UTF-8"
jobs_response = response.json()
print('Der Job wurde gestartet')

<Response [401]> {"Code":15,"Content":"Sie sind nicht berechtigt diesen Service aufzurufen! (KDTYP)","Type":"ERROR"}
Der Job wurde gestartet


In [9]:
jobs_response

{'Code': 15,
 'Content': 'Sie sind nicht berechtigt diesen Service aufzurufen! (KDTYP)',
 'Type': 'ERROR'}

Der Informations-Code 99 zeigt an, dass der Job angestoßen wurde. Die ID des Jobs lässt sich im Content des Status finden.

In [81]:
if jobs_response.get('Status').get('Code') == 99:
    s = jobs_response.get('Status').get('Content')
    job_ID = s.split(':')[1].strip()
    print('Der Job wurde angestoßen mit der ID: {}'.format(job_ID))

Der Job wurde angestoßen mit der ID: 42153-0001_604489979


Wir können den Status des Jobs über den 'catalgoue' Endpunkt abfragen.

In [93]:
params_catalogue = {
    'selection':job_ID,
    'sortcriterion':'date',
    'area':'Öffentlich',
    'pagelength':20,
    'language':'de',
}
catalogue_response = get_response_from_endpoint('catalogue', 'jobs?', params)

In [95]:
for elem in catalogue_response.json().get('List'):
    if elem.get('Code') == job_ID:
        job_result = elem
        print(job_result)
        print('Der Endpunkt catalogue/jobs wurde angesprochen. Der Status ist: {}'.format(job_result.get('State')))

{'Content': 'Werteabruf', 'Date': '27.08.2022', 'Time': '14:48:01', 'State': 'Fertig', 'Code': '42153-0001_604489979'}
Der Endpunkt catalogue/jobs wurde angesprochen. Der Status ist: Fertig


In [96]:
if job_result.get('State') != 'Fertig':
    print('Es wird auf die Fertigstellung des Jobs gewartet!'
          '\nBitte teste diesen Block später erneut.')
else:
    if job_ID == job_result.get('Code'):
        params_resultfile = {
            'name':job_ID,
            'area':'all',
            'compress':'false',
            'format':'csv',
            'language':'de',
        }
        resultfile = get_response_from_endpoint('data', 'resultfile?', params)
        resultfile = requests.get('{0}/data/resultfile?username={1}&password={2}&name={3}&area=all&compress=false&format=csv&language=de'.format(genesis_url, genesis_user, genesis_pwd,job_ID))
        print('Das Resultatedokument der ID {0} & wurde in folgender Variable gespeichert \'resultfile\''.format(job_ID))

Das Resultatedokument der ID 42153-0001_604489979 & wurde in folgender Variable gespeichert 'resultfile'
